**Jupyter Notebook to, given a set of embeddings, compute their cosine similarity _(in this case, of the answers-ground truth topic label pairs)_.**

In [2]:
# Import libraries
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from os import walk,rename
import torch

1. Read the topic ground truth

In [3]:
# Read the topics (names and descriptions)
topics = pd.read_json("topics_20news.json")

# Extract the topics' names and define a results dataframe to save similarities
results = pd.DataFrame({'topics':topics['name']})
# Show topics' names
print(results)

There are two similarities that can be generated:
- Similarities of conversational answers
- Similarities of QA answers

In the following code we show both methods

# CONV. SIMILARITIES

2a. For the conversational models, embedding model,question structure and number of topic words selected, generate the similarity values between the topic ground truth embedding and the conversational answer embedding.

In [9]:
# Define the name of conversational models
#conv_models = ["blenderbot-400M-distill","DialoGPT-medium","ChatGPT"]
conv_models = ["bart-tl-all","bart-tl-ng"]

# Define the embedding model name
#emb_model = "all-mpnet-base-v2"
emb_model = "all-MiniLM-L6-v2"

# Define the path where the embeddings of the topics are stored
emb_topic_path = "Bart-TL/BartEmbeddings/"+emb_model+"/topics/"

# Define the name of the question structure used
q = "q3"

# Define the number of words to be used
num = [4,10]
# For each number of words
for i in num:
    # Define the path where the embeddings of the answers are stored
    emb_path = "Bart-TL/BartEmbeddings/"+emb_model+"/"
    # Define the path where the similarities are to be stored
    sim_path = "Bart-TL/Similarities/"
    
    # For each conversational model
    for c in conv_models:
        # Array to save the similarities
        sims = []
        # Define the prefix name of the similarities' files of the conversational model c with i words
        #prefix = "tensor_"+q+"_topics_results_"+str(i)+"_"+c+"_"
        prefix = "tensor_topics_results_"+str(i)+"_"+c+"_"
        # For each topic (we have 20 topics)
        for j in range(0,20):
            # Define the path of the file where the conversational answer tensor for topic j is
            t_answ_name = emb_path+prefix+str(j)+".pt"
            # Define the path of the file where the groud truth tensor for topic j is
            t_topic_name = emb_topic_path+"tensor_topic_"+str(j)+".pt"
            # Load the conv. answer and GT tensors
            emb_answ = torch.load(t_answ_name)
            emb_topic = torch.load(t_topic_name)

            # Compute similarity scores of the two embeddings/tensors
            cosine_scores = util.pytorch_cos_sim(emb_topic, emb_answ)
            # Save the similarity value
            sims.append(cosine_scores[0][0].item())
        # Store the similarities by id topic
        d = {"id":range(0,20),"cosine_sim":sims}
        df = pd.DataFrame(data=d)
        # Save the results in a json file
        df.to_json(sim_path+q+"_topics_results_"+str(i)+"_"+emb_model+"_"+c+".json")
        # Save the results for each combination of conv. model and num. of words in a 'results' dataframe
        results[str(i)+"_"+c] = sims

2b. Show the similarity results obtained and save the results in a JSON file

In [10]:
results

,topics,4_bart-tl-all,4_bart-tl-ng,10_bart-tl-all,10_bart-tl-ng
0,sport hockey,0.455012,0.455012,0.053746,0.210210
1,religion atheism,0.586220,0.191222,0.536015,0.136997
2,science space,0.350002,0.350003,0.350003,0.350003
3,science medicine,0.252070,0.190956,0.168083,0.279395
4,politics_misc,0.226567,0.226567,0.331462,0.185002
5,computer mac hardware,0.433676,0.387201,0.387201,0.387201
6,politics mideast,0.261565,0.113921,0.362944,0.113921
7,computer ibm hardware,0.436425,0.423988,0.329375,0.423988
8,for sale,0.162414,0.404935,0.404935,0.404935
9,science electronics,0.055412,0.312479,0.162447,0.323538


In [8]:
# Save all the results as an xlsx file
results.to_excel("Bart-TL/Bart_Evaluation_"+emb_model+".xlsx")

# QA SIMILARITIES

3a. For the conversational models, QA models, embedding model, question structure and number of topic words selected, generate the similarity values between the topic ground truth embedding and the QA answer embedding.

In [22]:
# Define the name of conversational models
conv_models = ["blenderbot-400M-distill","DialoGPT-medium","ChatGPT"]

# Define the name of QA models
#qa_models = ["deberta-v3-large-squad2","deberta-v3-base-squad2", "xlm-roberta-large-squad2",\
#              "bert-large-uncased-whole-word-masking-squad2", "roberta-base-squad2-distilled"]
qa_models = ["deberta-v3-base-squad2", "bert-large-uncased-whole-word-masking-squad2", "roberta-base-squad2-distilled"]

# Define the embedding model name
#emb_model = "all-mpnet-base-v2"
emb_model = "all-MiniLM-L6-v2"

# Define the path where the embeddings of the topics are stored
emb_topic_path = "Eval/QAEmbeddings/"+emb_model+"/topics/"

# Define the name of the question structure used
q = "q3"

# Define the number of words to be used
num = [4,9,10]
# For each QA model
for qa in qa_models:
    # For each number of words
    for i in num:
        # Define the path where the embeddings of the answers are stored
        emb_path = "Eval/QAEmbeddings/"+emb_model+"/"
        # Define the path where the similarities are to be stored
        sim_path = "Eval/QASimilarities/"
        
        # For each conversational model
        for c in conv_models:
             # Array to save the similarities
            sims = []
            # Define the prefix name of the similarities' files of the conversational model c, QA model qa with i words
            prefix = "tensor_"+q+"_topics_results_"+str(i)+"_"+c+"_"+qa+"_"
            # For each topic (we have 20 topics)
            for j in range(0,20):
                # Define the path of the file where the QA answer tensor for topic j is
                t_answ_name = emb_path+prefix+str(j)+".pt"
                 # Define the path of the file where the groud truth tensor for topic j is
                t_topic_name = emb_topic_path+"tensor_topic_"+str(j)+".pt"
                # Load the conv. answer and GT tensors
                emb_answ = torch.load(t_answ_name)
                emb_topic = torch.load(t_topic_name)

                # Compute similarity scores of the two embeddings/tensors
                cosine_scores = util.pytorch_cos_sim(emb_topic, emb_answ)
                # Save the similarity value
                sims.append(cosine_scores[0][0].item())

            # Store the similarities by id topic
            d = {"id":range(0,20),"cosine_sim":sims}
            df = pd.DataFrame(data=d)
             # Save the results in a json file
            df.to_json(sim_path+q+"_topics_results_"+str(i)+"_"+emb_model+"_"+c+"_"+qa+".json")
             # Save the results for each combination of conv. model, QA model and num. of words in a 'results' dataframe
            results[str(i)+"_"+c+"_"+qa] = sims



3b. Show the similarity results obtained and save the results in a JSON file

In [ ]:
results

In [19]:
# Save all the results as an xlsx file
results.to_excel("Eval/QA_Evaluation_"+emb_model+".xlsx")